Melbourne Housing Dataset is a common dataset every Kaggler goes through when they start with the introductory machine learning course. But the dataset has one issue that is not addressed in the course, and with good reasons, and that is too many categorical features. Handling categorical features is taught in the Intermediate course.  

This dataset has so many categorical features and some of the features have too many categories. And in the introductory course, only numerical features are used to keep things simple. So, in this notebook, I've demonstrated the use of One Hot Encoding and Target Encoding for other categorical features.

# Objective

The objective of this notebook is to demonstrate how One Hot Encoding and Target Encoding can be used to convert Categorical features to Numerical features so that they can be further used for other purposes to make our model better. 

The objective in this notebook is not to get the best result. Getting best result will take a lot more than encoding, and more after encoding(for eg. checking correlations, smoothing, tuning hyperparameters etc.). 

# Setup and Import Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melbourne-housing-snapshot/melb_data.csv


# Load The Dataset

In [2]:
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
df = pd.read_csv(melbourne_file_path)

#let's get rid of empty rows first
df = df.dropna(axis=0)
# reseting the index helps to concat dataframes more easily. We'll need that later
df = df.reset_index(drop=True)
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
1,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
2,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
3,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
4,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


In [3]:
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

From the list above, Suburb, Address, Type, Method, SellerG, Date, Postcode, CouncilArea, Regionname are categorical features.

'Postcode' has numerical values and technically can be used in an ML model without an error but it would give you an inappropriate result. Because, it represents a category. It has no numerical significance. 

# Working With Just The Numerical Features

Let's see the result when we use just the basic numerical features that work with Random Forest. And later we will see whether adding those categorical features will actually improve our model or make it worse. 

In [4]:
features = ['Rooms','Distance','Bedroom2','Bathroom', 'Car','Landsize',
            'BuildingArea','Lattitude','Longtitude','Propertycount']
y = df.Price
X = df[features]
train_X, test_X, train_y, test_y = train_test_split(X,y, random_state =1)
forest_model = RandomForestRegressor(random_state=1, n_estimators = 100 )
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(test_X)
print(mean_absolute_error(test_y, melb_preds))

175105.8469491838


Now, let's work with the categorical data and add them into our features list. 

# One Hot Encoding 
Here's a quick introduction of One Hot Encoding: https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/.

In short, the categories are converted into columns. For example, in our 'Type' column, there are three categories, so we get three columns. And the rows are filled with 0's and 1's. The final result is a matrix of 0's and 1's where presence of each category is represented with 1. 

One Hot Encoding is only good for features that have few categories. With too many categories, we will be cursed with the 'Curse of Dimensionality'. In our dataset, it is only good for 'Type' and 'Method'. All the others have too many categories. 

In [5]:
# create the encoder
ohe = OneHotEncoder(handle_unknown = 'ignore')

# fit the encoder with the categorical data 
encoded_columns = ohe.fit_transform(df[['Type','Method']])

# after fitting the data to the encoder, we can view the categories inside Type and Method, and the result data
# the resulting data will make more sense after you change it into an actual dataframe which we'll do later
print(ohe.categories_,'\n',encoded_columns.toarray())

# we will use the categories names as column names to add to the dataframe
column_names = np.concatenate([ohe.categories_[0],ohe.categories_[1]],axis = 0)

# Now let's create the dataframe 
encoded_df = pd.DataFrame(encoded_columns.toarray(),columns = column_names)
encoded_df

# we will concatenate this dataframe to our original dataframe
df = pd.concat([df,encoded_df],axis =1)
df.head()

[array(['h', 't', 'u'], dtype=object), array(['PI', 'S', 'SA', 'SP', 'VB'], dtype=object)] 
 [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]]


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Regionname,Propertycount,h,t,u,PI,S,SA,SP,VB
0,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,Northern Metropolitan,4019.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,Northern Metropolitan,4019.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,Northern Metropolitan,4019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,...,Northern Metropolitan,4019.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,...,Northern Metropolitan,4019.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


We have successfully converted two categorical features to numerical features. But this process has also added 6 more columns. This is the Curse of Dimensionality. More categories would result in more columns. That is why One Hot Encoding is not suitable for features with large number of categories. It will enlarge the dataset by a lot. That is where Target Encoding comes in. 

# Target Encoding

With target encoding, we replace the categories with their respective mean values of target data. For example let's just take the feature 'Suburb' right now and check mean values for all the categories inside. 

In [6]:
suburb_means = df.groupby('Suburb')['Price'].mean()
suburb_means

Suburb
Abbotsford      1.125972e+06
Aberfeldie      1.365000e+06
Airport West    7.042941e+05
Albanvale       5.550000e+05
Albert Park     1.868783e+06
                    ...     
Wollert         5.625000e+05
Wyndham Vale    4.860000e+05
Yallambie       8.646786e+05
Yarra Glen      6.200000e+05
Yarraville      1.018802e+06
Name: Price, Length: 287, dtype: float64

These mean values will replace the actual categories in 'Suburb' feature. For instance, Abbotsford will be replaced by 1.125972e+06. Now, let's see the actual implementation. 

In [7]:
suburb_means = df.groupby('Suburb')['Price'].mean()
postcode_means = df.groupby('Postcode')['Price'].mean()
sellerg_means = df.groupby('SellerG')['Price'].mean()
council_area_means = df.groupby('CouncilArea')['Price'].mean()
region_name_means = df.groupby('Regionname')['Price'].mean()
df['Suburb'] = df['Suburb'].map(suburb_means)
df['Postcode'] = df['Postcode'].map(postcode_means)
df['SellerG'] = df['SellerG'].map(sellerg_means)
df['CouncilArea'] = df['CouncilArea'].map(council_area_means)
df['Regionname'] = df['Regionname'].map(region_name_means)
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Regionname,Propertycount,h,t,u,PI,S,SA,SP,VB
0,1.125972e+06,25 Bloomburg St,2,h,1035000.0,S,1.027889e+06,4/02/2016,2.5,1.125972e+06,...,872263.457929,4019.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.125972e+06,5 Charles St,3,h,1465000.0,SP,1.027889e+06,4/03/2017,2.5,1.125972e+06,...,872263.457929,4019.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.125972e+06,55a Park St,4,h,1600000.0,VB,1.011728e+06,4/06/2016,2.5,1.125972e+06,...,872263.457929,4019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.125972e+06,124 Yarra St,3,h,1876000.0,S,1.011728e+06,7/05/2016,2.5,1.125972e+06,...,872263.457929,4019.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.125972e+06,98 Charles St,2,h,1636000.0,S,1.011728e+06,8/10/2016,2.5,1.125972e+06,...,872263.457929,4019.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


We are almost done. Now we have two categorical features that we haven't touched, 'Address' and 'Date'. We will ignore the 'Address' column because it just has way too many categories to address (pun intended). From 'Date' column, we will extract the year and get how many years ago it was sold.

# Working with Dates

Apart from the obvious 'Date' column, there's also the column 'YearBuilt'. We could leave YearBuilt because it does have numerical significance and works fine. But it would be more appropriate to extract how old the house exactly is and use that instead. So, we're doing that as well. 

In [8]:
from datetime import date

current_year = date.today().year

# converting the Date column from String Object to datetime datatype
df['Date'] = pd.to_datetime(df['Date'],dayfirst = True)

# how many years ago was it sold ? 
sold_years_ago = [current_year - date.year for date in df['Date']]

# replacing the Date column with sold_years_ago data
df['sold_years_ago'] = sold_years_ago


# getting how old the house is from YearBuilt column
years_old = [current_year - year for year in df.YearBuilt]
df['years_old'] = years_old

# Final Dataset

We are done working with all the data. Now, let's check out the final dataset.

In [9]:
print(df.head(),'\n', df.columns, '\n')

         Suburb          Address  Rooms Type      Price Method       SellerG  \
0  1.125972e+06  25 Bloomburg St      2    h  1035000.0      S  1.027889e+06   
1  1.125972e+06     5 Charles St      3    h  1465000.0     SP  1.027889e+06   
2  1.125972e+06      55a Park St      4    h  1600000.0     VB  1.011728e+06   
3  1.125972e+06     124 Yarra St      3    h  1876000.0      S  1.011728e+06   
4  1.125972e+06    98 Charles St      2    h  1636000.0      S  1.011728e+06   

        Date  Distance      Postcode  ...    h    t    u   PI    S   SA   SP  \
0 2016-02-04       2.5  1.125972e+06  ...  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
1 2017-03-04       2.5  1.125972e+06  ...  1.0  0.0  0.0  0.0  0.0  0.0  1.0   
2 2016-06-04       2.5  1.125972e+06  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
3 2016-05-07       2.5  1.125972e+06  ...  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
4 2016-10-08       2.5  1.125972e+06  ...  1.0  0.0  0.0  0.0  1.0  0.0  0.0   

    VB  sold_years_ago  years_old  
0 

Looking at the columns, we don't need some of them anymore. We can drop Address, Type, Method, Date and YearBuilt. 

In [10]:
df = df.drop(['Address','Type','Method','Date','YearBuilt'],axis = 1)

# now, lets take a look at the final dataset 
df.head()

,Suburb,Rooms,Price,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,...,h,t,u,PI,S,SA,SP,VB,sold_years_ago,years_old
0,1.125972e+06,2,1035000.0,1.027889e+06,2.5,1.125972e+06,2.0,1.0,0.0,156.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7,123.0
1,1.125972e+06,3,1465000.0,1.027889e+06,2.5,1.125972e+06,3.0,2.0,0.0,134.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6,123.0
2,1.125972e+06,4,1600000.0,1.011728e+06,2.5,1.125972e+06,3.0,1.0,2.0,120.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,9.0
3,1.125972e+06,3,1876000.0,1.011728e+06,2.5,1.125972e+06,4.0,2.0,0.0,245.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7,113.0
4,1.125972e+06,2,1636000.0,1.011728e+06,2.5,1.125972e+06,2.0,1.0,2.0,256.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7,133.0


# Using The Final Dataset

Let's use all the data and see the result. 

In [11]:
# making training and testing dataset
X = df.loc[:,df.columns != 'Price']
y = df.Price
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 1)
reg = RandomForestRegressor(random_state = 1,n_estimators = 100).fit(train_X,train_y)
predicted_y = reg.predict(test_X)
mean_absolute_error(test_y, predicted_y)

164897.82257800733

# Conclusion

The result is a little bit better. We could definitely improve it  by selecting only some useful features, tuning the hyperparameters for the model, smoothing the target encoded data and maybe few other processes. In this notebook, we just saw how we can implement One Hot and Target encoding to handle categorical data.  